In [1]:
from selenium import webdriver

In [2]:
from bs4 import BeautifulSoup

In [19]:
import os
import requests

In [4]:
URL = "https://wbes.nrldc.in/xml/"

In [5]:
page = requests.get(URL)

In [30]:
#print(page.text)

In [6]:
soup = BeautifulSoup(page.content, "html.parser")

In [7]:
#results = soup.find(id="ResultsContainer")
filename=[]
dates=[]
for tr in soup.findAll('tr'):
    for td in tr.findAll('td'):
        if(td.find('a',href=True)):
            link=td.find('a',href=True)
            filename.append(link.text)
        else :
            dates.append(td.text)
        
                
#print(filename)

In [8]:
import pandas as pd
df = pd.DataFrame({'File':filename,'Time':dates}) 
#df.to_csv('D://nrldc.csv', index=False, encoding='utf-8')

In [11]:
df.head(5)

,File,Time
0,FullSchedule-(171)-27-10-2021.xml,10/27/2021 5:52:58 PM
1,FullSchedule-(170)-27-10-2021.xml,10/27/2021 5:49:07 PM
2,FullSchedule-(169)-27-10-2021.xml,10/27/2021 5:38:07 PM
3,FullSchedule-(168)-27-10-2021.xml,10/27/2021 5:34:19 PM
4,FullSchedule-(167)-27-10-2021.xml,10/27/2021 5:23:23 PM


In [14]:
from datetime import datetime
rawtable = pd.read_html(URL)

In [16]:
df = pd.DataFrame(rawtable[0])

df["Timestamp"] = pd.to_datetime(df.Timestamp, format="%m/%d/%Y %I:%M:%S %p")
df.head(5)

,Filename,Timestamp
0,FullSchedule-(2)-28-10-2021.xml,2021-10-27 18:05:00
1,FullSchedule-(172)-27-10-2021.xml,2021-10-27 18:04:12
2,FullSchedule-(171)-27-10-2021.xml,2021-10-27 17:52:58
3,FullSchedule-(170)-27-10-2021.xml,2021-10-27 17:49:07
4,FullSchedule-(169)-27-10-2021.xml,2021-10-27 17:38:07


In [22]:
for index, row in rawtable:
    print(index)
    print(row)
    #print("Downloading Record "+str(row.Filename))
    # print(str(row.Filename))
    download_url = URL + row
    print(download_url)

    response = requests.get(download_url)
    print(response)

    folderpath = r"F:\flutegrid\NRLDC_Python_Example\NRLDC"
    filename = os.path.join(folderpath, row)
    print(filename)
    #print(row[2:])

    #print(response.content)

    with open(filename, 'wb') as file:
        file.write(response.content)
        #print(file)
    print("Done")

Filename
Timestamp
https://wbes.nrldc.in/xml/Timestamp
<Response [404]>
F:\flutegrid\NRLDC_Python_Example\NRLDC\Timestamp
Done


In [34]:
import pyodbc

In [35]:
from datetime import datetime
server = '10.10.69.97' 
database = 'NTPCPowerBI' 
username = 'sa' 
password = 'Ntpc@123' 
date_time_obj=datetime.strptime('1/1/2021 10:49:05 AM', '%m/%d/%Y %I:%M:%S %p')
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
cursor.execute("select top 1 FILENAME,FILETIME from NTPCPowerBI.dbo.RLDC_XML_PortingStatus  with (nolock) order by FILETIME desc")
for row in cursor.fetchall():
    date_time_str=row.FILETIME
    print(date_time_str)
    #date_time_obj = datetime.strptime(date_time_str, '%m/%d/%Y %I:%M:%S %p')
    date_time_obj = date_time_str  

cnxn.commit()
cursor.close()

In [36]:
server = '10.10.69.97' 
database = 'NTPCPowerBI' 
username = 'sa' 
password = 'Ntpc@123' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
for index, row in df.iterrows():
    web_date_time_str=row.Time.strip()
    web_url=row.File.strip()
    web_date_time_obj = datetime.strptime(web_date_time_str, '%m/%d/%Y %I:%M:%S %p')
    if(web_date_time_obj>date_time_obj):
        cursor.execute("INSERT INTO NTPCPowerBI.dbo.RLDC_XML_PortingStatus (FileName,FileTime,RLDC,Porting_status) values(?,?,'NRLDC','N')", web_url, web_date_time_str)
        download_url = "https://wbes.nrldc.in/xml/"+web_url
        print(download_url +" --- "+ web_date_time_str)
        #r = requests.get(download_url)
        #with open("D://"+web_url,'wb') as f:
         # f.write(r.content)
cnxn.commit()
cursor.close()

https://wbes.nrldc.in/xml/FullSchedule-(117)-09-07-2021.xml --- 7/9/2021 12:04:18 PM
https://wbes.nrldc.in/xml/FullSchedule-(116)-09-07-2021.xml --- 7/9/2021 11:53:09 AM
https://wbes.nrldc.in/xml/FullSchedule-(115)-09-07-2021.xml --- 7/9/2021 11:49:24 AM
https://wbes.nrldc.in/xml/FullSchedule-(113)-09-07-2021.xml --- 7/9/2021 11:34:03 AM
https://wbes.nrldc.in/xml/FullSchedule-(112)-09-07-2021.xml --- 7/9/2021 11:23:16 AM
https://wbes.nrldc.in/xml/FullSchedule-(111)-09-07-2021.xml --- 7/9/2021 11:18:55 AM
https://wbes.nrldc.in/xml/FullSchedule-(1)-10-07-2021.xml --- 7/9/2021 11:08:37 AM
https://wbes.nrldc.in/xml/FullSchedule-(110)-09-07-2021.xml --- 7/9/2021 11:08:07 AM
https://wbes.nrldc.in/xml/FullSchedule-(109)-09-07-2021.xml --- 7/9/2021 11:03:56 AM
https://wbes.nrldc.in/xml/FullSchedule-(108)-09-07-2021.xml --- 7/9/2021 10:53:15 AM
https://wbes.nrldc.in/xml/FullSchedule-(107)-09-07-2021.xml --- 7/9/2021 10:49:10 AM
https://wbes.nrldc.in/xml/FullSchedule-(106)-09-07-2021.xml --- 7/9